In [1]:
import os, sys, time
ROOT_PATH = r"D:\YuWei\ML_DL\_Courses\HungYiLee\ML_2019_HW"
os.chdir(ROOT_PATH)
sys.path.append(ROOT_PATH)

from keras.datasets import cifar10
# from practice.src import model as modellib
import practice.src.model as modelib
import practice.src.utils as utils
from practice.src.config import Config
import practice.cifar10_keras as runfile
import numpy as np

DEFAULT_LOG_PATH = os.path.join(ROOT_PATH, "logs")

############################################################
#  Session Setting
############################################################
# If you face the error about convolution layer,
# use this block to enable the memory usage of GPU growth.
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=tf_config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

Using TensorFlow backend.


In [3]:
class Cifar10Config(Config):
    # The name of your work:
    NAME = 'cifar10_test'

    # Number of class in dataset:
    N_CLASS = 10

    # The shape of image in dataset:
    IMG_SHAPE = [32, 32, 3]

    # Learning rate:
    LR = 0.001

    # Epochs:
    EPOCHS = 20

    # Training batch size:
    BATCH_SIZE = 256

    # Validation rate:
    VALIDATION_RATE = 0.2
    
class ClassifyConfig(Cifar10Config):
    BATCH_SIZE = 1
config = ClassifyConfig()
config_list = config.display(True)

Configurations:
ACTIVATION_FUNC                relu
BATCH_SIZE                     1
BIAS_INIT_DEFAULT              0
CONV2D_KERNEL_SIZE             (3, 3)
CONV2D_PADDING                 SAME
CONV2D_STRIDES                 1
DROPOUT_RATE                   0
EPOCHS                         20
IMG_SHAPE                      [32, 32, 3]
KERNEL_INIT_METHOD             glorot_normal
LR                             0.001
MAXPOOL2D_KERNEL_SIZE          (2, 2)
MAXPOOL2D_PADDING              SAME
MAXPOOL2D_STRIDES              2
NAME                           cifar10_test
N_CLASS                        10
SAVE_BEST_ONLY                 False
UPSAMPLING_KERNEL_SIZE         2
UPSAMPLING_METHOD              nearest
VALIDATION_RATE                0.2


In [4]:
config_list

['Configurations:',
 'ACTIVATION_FUNC                relu',
 'BATCH_SIZE                     1',
 'BIAS_INIT_DEFAULT              0',
 'CONV2D_KERNEL_SIZE             (3, 3)',
 'CONV2D_PADDING                 SAME',
 'CONV2D_STRIDES                 1',
 'DROPOUT_RATE                   0',
 'EPOCHS                         20',
 'IMG_SHAPE                      [32, 32, 3]',
 'KERNEL_INIT_METHOD             glorot_normal',
 'LR                             0.001',
 'MAXPOOL2D_KERNEL_SIZE          (2, 2)',
 'MAXPOOL2D_PADDING              SAME',
 'MAXPOOL2D_STRIDES              2',
 'NAME                           cifar10_test',
 'N_CLASS                        10',
 'SAVE_BEST_ONLY                 False',
 'UPSAMPLING_KERNEL_SIZE         2',
 'UPSAMPLING_METHOD              nearest',
 'VALIDATION_RATE                0.2']

In [3]:
weights_path = "./practice/logs/D20190930T0035/cifar10_test_epoch020.h5"
model = modelib.NNModel(mode="classify", config=config, logdir=None, resume=0)
model.load_weights(weights_path)

Instructions for updating:
Colocations handled automatically by placer.
Name                      | Input_shape               | Kernel size          | Strides  | Padding  | Output_shape        
-------------------------------------------------------------------------------------------------------------------------
conv2d_1                  | (None, 32, 32, 3)         | (5, 5, 3, 64)        | (1, 1)   | same     | (None, 32, 32, 64)  
batch_normalization_1     | (None, 32, 32, 64)        |                      |          |          |                     
activation_1/Relu:0       |                           |                      |          |          |                     
max_pooling2d_1           | (None, 32, 32, 64)        | (2, 2)               | (2, 2)   | same     | (None, 16, 16, 64)  
conv2d_2                  | (None, 16, 16, 64)        | (5, 5, 64, 128)      | (1, 1)   | same     | (None, 16, 16, 128) 
batch_normalization_2     | (None, 16, 16, 128)       |                   

In [4]:
class Cifar10Dataset(utils.Dataset):
    def feed_config(self, config):
        self.n_class = config.N_CLASS
        self.img_shape = config.IMG_SHAPE
        self.val_rate = config.VALIDATION_RATE

    def load(self, usage):
        if usage == 'train':
            (self.x_data, self.y_data), _ = cifar10.load_data()
            total = self.x_data.shape[0]
            self.x_data = self.x_data[0: int(total * (1 - self.val_rate))]
            self.y_data = self.y_data[0: int(total * (1 - self.val_rate))]
        elif usage == 'val':
            (self.x_data, self.y_data), _ = cifar10.load_data()
            total = self.x_data.shape[0]
            self.x_data = self.x_data[int(total * (1 - self.val_rate)): ]
            self.y_data = self.y_data[int(total * (1 - self.val_rate)): ]
        else:
            _, (self.x_data, self.y_data) = cifar10.load_data()

In [5]:
dataset_test = Cifar10Dataset()
dataset_test.feed_config(config)
dataset_test.load('test')
dataset_test.prepare()

In [6]:
prediction = model.classify(dataset_test.x_data)

10000/10000 [==============================] - 3s 272us/step
Saved prediction to ./practice/logs/D20190930T0035/classify_cifar10_test_epoch020.csv


In [7]:
accuracy = model.evaluate(prediction['label'].values, np.argmax(dataset_test.y_data, axis=1))

Test accuracy: 0.7738
